In [3]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import sklearn

import aif360
from aif360.datasets import AdultDataset, BankDataset, CompasDataset, GermanDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms import preprocessing, inprocessing, postprocessing
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
            import load_preproc_data_adult, load_preproc_data_german, load_preproc_data_compas
from aif360.algorithms.preprocessing.optim_preproc_helpers.distortion_functions\
            import get_distortion_adult, get_distortion_german, get_distortion_compas
from aif360.algorithms.preprocessing.optim_preproc import OptimPreproc
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools

import art

import copy

from IPython.display import Markdown, display
import warnings
warnings.filterwarnings('ignore')
%load_ext jupyternotify
np.random.seed(1)

<IPython.core.display.Javascript object>

In [4]:
def run_classification_metrics(CM:ClassificationMetric):
    return np.array([
        round(CM.accuracy(), 4),
        round(CM.theil_index(), 4),
        round(CM.consistency()[0], 4),
        round(CM.false_positive_rate_difference(), 4),
        round(CM.false_negative_rate_difference(), 4),
        round(-CM.error_rate_difference(), 4),
        round(CM.false_discovery_rate_difference(), 4),
        round(CM.false_omission_rate_difference(), 4)
    ])

In [5]:
def run_binary_dataset_metrics(BLDM:BinaryLabelDatasetMetric):
    return np.array([
        round(BLDM.statistical_parity_difference(), 4), # negative means privileged bias
        round(BLDM.base_rate(privileged=True), 4), # 1 means privileged bias
        round(BLDM.base_rate(privileged=False), 4), # 1 means unprivileged bias
    ])

In [15]:
def get_comparison_algo(inprocessing_algo):
    if isinstance(inprocessing_algo, inprocessing.PrejudiceRemover):
        return sklearn.linear_model.LogisticRegression()
    if isinstance(inprocessing_algo, inprocessing.GerryFairClassifier):
        return sklearn.linear_model.LogisticRegression()
    if isinstance(inprocessing_algo, inprocessing.MetaFairClassifier):
        return BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)
    if isinstance(inprocessing_algo, inprocessing.ExponentiatedGradientReduction):
        return sklearn.linear_model.LogisticRegression()
    if isinstance(inprocessing_algo, inprocessing.GridSearchReduction):
        return sklearn.linear_model.LogisticRegression()

In [7]:
def get_model_name(model):
    if isinstance(model, sklearn.linear_model.LogisticRegression):
        return "Logistic Regression"
    if isinstance(model, sklearn.linear_model.LinearRegression):
        return "Linear Regression"
    if isinstance(model, sklearn.ensemble.BaggingClassifier):
        return "Meta Classifier"
    
    if isinstance(model, preprocessing.DisparateImpactRemover):
        return "Disparate Impact Remover"
    if isinstance(model, preprocessing.LFR):
        return "Learning Fair Representations"
    if isinstance(model, preprocessing.OptimPreproc):
        return "Optimized Preprocessing"
    if isinstance(model, preprocessing.Reweighing):
        return "Reweighing"
    
    if isinstance(model, inprocessing.PrejudiceRemover):
        return "Prejudice Remover"
    if isinstance(model, inprocessing.AdversarialDebiasing):
        return "Adversarial Debiasing"
    if isinstance(model, inprocessing.ARTClassifier):
        return "ART Classifier"
    if isinstance(model, inprocessing.ExponentiatedGradientReduction):
        return "Exp Grad Reduction"
    if isinstance(model, inprocessing.GerryFairClassifier):
        return "GerryFair Classifier"
    if isinstance(model, inprocessing.GridSearchReduction):
        return "GridSearch Reduction"
    if isinstance(model, inprocessing.MetaFairClassifier):
        return "MetaFair Classifier"
    
    if isinstance(model, postprocessing.EqOddsPostprocessing):
        return "Eq Odds Post."
    if isinstance(model, postprocessing.CalibratedEqOddsPostprocessing):
        return "Calibrated Eq Odds Post."
    if isinstance(model, postprocessing.RejectOptionClassification):
        return "RejectOption Classification"
    
    return "None"

In [8]:
def get_dataset_name(dataset):
    if isinstance(dataset, aif360.datasets.german_dataset.GermanDataset):
        return "German Dataset"
    if isinstance(dataset, aif360.datasets.adult_dataset.AdultDataset):
        return "Adult Dataset"
    if isinstance(dataset, aif360.datasets.bank_dataset.BankDataset):
        return "Bank Dataset"
    if isinstance(dataset, aif360.datasets.compas_dataset.CompasDataset):
        return "Compas Dataset"

In [9]:
def get_optim_preproc_options_dict(dataset):
    if isinstance(dataset, aif360.datasets.german_dataset.GermanDataset):
        optim_options = {
            "distortion_fun": get_distortion_german,
            "epsilon": 0.05,
            "clist": [0.99, 1.99, 2.99],
            "dlist": [.1, 0.05, 0]
        }
    elif isinstance(dataset, aif360.datasets.adult_dataset.AdultDataset):
        optim_options = {
            "distortion_fun": get_distortion_adult,
            "epsilon": 0.05,
            "clist": [0.99, 1.99, 2.99],
            "dlist": [.1, 0.05, 0]
        }
    elif isinstance(dataset, aif360.datasets.bank_dataset.BankDataset):
        optim_options = {
            "distortion_fun": get_distortion_bank,
            "epsilon": 0.05,
            "clist": [0.99, 1.99, 2.99],
            "dlist": [.1, 0.05, 0]
        }
    elif isinstance(dataset, aif360.datasets.compas_dataset.CompasDataset):
        optim_options = {
            "distortion_fun": get_distortion_compas,
            "epsilon": 0.05,
            "clist": [0.99, 1.99, 2.99],
            "dlist": [.1, 0.05, 0]
        }
    return optim_options

In [10]:
def run_base(dataset_train, dataset_test, model):
    model.fit(dataset_train.features, dataset_train.labels.ravel())
    results = model.predict(dataset_test.features)
    
    if isinstance(model, sklearn.linear_model.LinearRegression):
        results = np.rint(results)
    dataset_test_pred = dataset_test.copy()
    dataset_test_pred.labels = np.array([results]).transpose()
    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    BLDM = BinaryLabelDatasetMetric(dataset_train,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    return np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))

In [20]:
def analyze_algo(dataset_train, dataset_test, privileged_groups, unprivileged_groups, classifier=None, 
                 preprocessing_algo=None, inprocessing_algo=None, postprocessing_algo=None):
    base = sklearn.linear_model.LogisticRegression()
    if inprocessing_algo is not None:
        base = get_comparison_algo(inprocessing_algo)
    '''base.fit(dataset_train.features, dataset_train.labels.ravel())
    results = base.predict(dataset_test.features)
    if isinstance(base, sklearn.linear_model.LinearRegression):
        results = np.rint(results)
    dataset_test_pred = dataset_test.copy()
    dataset_test_pred.labels = np.array([results]).transpose()'''
    
    dataset_train_pred = dataset_train.copy(deepcopy=True)
    dataset_test_pred = dataset_test.copy(deepcopy=True)
    
    scale_orig = StandardScaler()
    X_train = scale_orig.fit_transform(dataset_train.features)
    y_train = dataset_train.labels.ravel()
    model = base
    model.fit(X_train, y_train)

    fav_idx = np.where(model.classes_ == dataset_train.favorable_label)[0][0]
    y_train_pred_prob = model.predict_proba(X_train)[:,fav_idx]

    # Prediction probs for testing data
    X_test = scale_orig.transform(dataset_test.features)
    y_test_pred_prob = model.predict_proba(X_test)[:,fav_idx]

    dataset_train_pred.scores = y_train_pred_prob.reshape(-1,1)
    dataset_test_pred.scores = y_test_pred_prob.reshape(-1,1)

    class_thresh = 0.5
    y_train_pred = np.zeros_like(dataset_train_pred.labels)
    y_train_pred[y_train_pred_prob >= class_thresh] = dataset_train_pred.favorable_label
    y_train_pred[~(y_train_pred_prob >= class_thresh)] = dataset_train_pred.unfavorable_label
    dataset_train_pred.labels = y_train_pred

    y_test_pred = np.zeros_like(dataset_test_pred.labels)
    y_test_pred[y_test_pred_prob >= class_thresh] = dataset_test_pred.favorable_label
    y_test_pred[~(y_test_pred_prob >= class_thresh)] = dataset_test_pred.unfavorable_label
    dataset_test_pred.labels = y_test_pred
    
    
    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    BLDM = BinaryLabelDatasetMetric(dataset_train,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    metrics = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))
    df = pd.DataFrame(metrics, columns=[get_model_name(base)])
    df.index = ["accuracy", "theil index", "consistency", "false positive rate difference",
                "false negative rate difference", "accuracy difference",
                "false discovery rate difference", "false omission rate difference",
                "stat parity difference", "priv base rate", "unpriv base rate"]
    
    if preprocessing_algo is not None:
        dataset_train = preprocessing_algo.fit_transform(dataset_train)
    
    if inprocessing_algo is not None:
        inprocessing_algo.fit(dataset_train)
        fair_results = inprocessing_algo.predict(dataset_test)
        dataset_test_pred = dataset_test.copy()
        dataset_test_pred.labels = fair_results.labels
        #dataset_test_pred = results
        
        dataset_train_pred = dataset_train.copy(deepcopy=True)
        dataset_test_pred = dataset_test.copy(deepcopy=True)

        scale_orig = StandardScaler()
        X_train = scale_orig.fit_transform(dataset_train.features)
        y_train = dataset_train.labels.ravel()
        inprocessing_algo.fit(dataset_train)

        fav_idx = np.where(model.classes_ == dataset_train.favorable_label)[0][0]
        y_train_pred_prob = inprocessing_algo.predict(dataset_train).favorable_label

        # Prediction probs for testing data
        X_test = scale_orig.transform(dataset_test.features)
        y_test_pred_prob = inprocessing_algo.predict(dataset_test)

        dataset_train_pred.scores = y_train_pred_prob.reshape(-1,1)
        dataset_test_pred.scores = y_test_pred_prob.reshape(-1,1)

        class_thresh = 0.5
        y_train_pred = np.zeros_like(dataset_train_pred.labels)
        y_train_pred[y_train_pred_prob >= class_thresh] = dataset_train_pred.favorable_label
        y_train_pred[~(y_train_pred_prob >= class_thresh)] = dataset_train_pred.unfavorable_label
        dataset_train_pred.labels = y_train_pred

        y_test_pred = np.zeros_like(dataset_test_pred.labels)
        y_test_pred[y_test_pred_prob >= class_thresh] = dataset_test_pred.favorable_label
        y_test_pred[~(y_test_pred_prob >= class_thresh)] = dataset_test_pred.unfavorable_label
        dataset_test_pred.labels = y_test_pred
    else:
        dataset_train_pred = dataset_train.copy(deepcopy=True)
        dataset_test_pred = dataset_test.copy(deepcopy=True)

        scale_orig = StandardScaler()
        X_train = scale_orig.fit_transform(dataset_train.features)
        y_train = dataset_train.labels.ravel()
        model = base
        model.fit(X_train, y_train)

        fav_idx = np.where(model.classes_ == dataset_train.favorable_label)[0][0]
        y_train_pred_prob = model.predict_proba(X_train)[:,fav_idx]

        # Prediction probs for testing data
        X_test = scale_orig.transform(dataset_test.features)
        y_test_pred_prob = model.predict_proba(X_test)[:,fav_idx]

        dataset_train_pred.scores = y_train_pred_prob.reshape(-1,1)
        dataset_test_pred.scores = y_test_pred_prob.reshape(-1,1)

        class_thresh = 0.5
        y_train_pred = np.zeros_like(dataset_train_pred.labels)
        y_train_pred[y_train_pred_prob >= class_thresh] = dataset_train_pred.favorable_label
        y_train_pred[~(y_train_pred_prob >= class_thresh)] = dataset_train_pred.unfavorable_label
        dataset_train_pred.labels = y_train_pred

        y_test_pred = np.zeros_like(dataset_test_pred.labels)
        y_test_pred[y_test_pred_prob >= class_thresh] = dataset_test_pred.favorable_label
        y_test_pred[~(y_test_pred_prob >= class_thresh)] = dataset_test_pred.unfavorable_label
        dataset_test_pred.labels = y_test_pred
    
    if postprocessing_algo is not None:
        dataset_test_pred = postprocessing_algo.fit_predict(dataset_test, dataset_test_pred)

    CM = ClassificationMetric(dataset_test,
                              dataset_test_pred,
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)
    BLDM = BinaryLabelDatasetMetric(dataset_train,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    name = ""
    if preprocessing_algo is not None:
        name += get_model_name(preprocessing_algo) + " + "
    if inprocessing_algo is not None:
        name += get_model_name(inprocessing_algo) + " + "
    if postprocessing_algo is not None:
        name += get_model_name(postprocessing_algo)
    df[name] = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))
    
    return df

In [18]:
def run_inproc_algo(dataset_train, dataset_test, unprivileged_groups, privileged_groups, inprocessing_algo, ):

    base = get_comparison_algo(inprocessing_algo)

    metrics = analyze_algo(dataset_train.copy(), dataset_test, privileged_groups, unprivileged_groups, base)
    df = pd.DataFrame(metrics, columns=[get_model_name(base)])
    df.index = ["accuracy", "theil index", "consistency", "false positive rate difference",
                "false negative rate difference", "accuracy difference",
                "false discovery rate difference", "false omission rate difference",
                "stat parity difference", "priv base rate", "unpriv base rate"]
    if isinstance(dataset, GermanDataset) and isinstance(inprocessing_algo, inprocessing.GridSearchReduction):
        dataset_train = dataset_train.copy()
        dataset_train.labels = dataset_train.labels%2
        dataset_test = dataset_test.copy()
        dataset_test.labels = dataset_test.labels%2
                  
    df[get_model_name(inprocessing_algo)] = analyze_algo(dataset_train, dataset_test, privileged_groups, unprivileged_groups, inprocessing_algo=inprocessing_algo)
    return df

## Combination Analysis

In [21]:
%%notify
dataset = CompasDataset()
privileged_groups = [{'race': 1}]
unprivileged_groups = [{'race': 0}]

dataset_train, dataset_test = dataset.split([0.7], shuffle = True)

preprocessing_algos = [preprocessing.Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups),
                      preprocessing.DisparateImpactRemover(),
                      None]
inprocessing_algos = [#inprocessing.ExponentiatedGradientReduction(sklearn.linear_model.LogisticRegression(), constraints="DemographicParity", drop_prot_attr=False),
                     inprocessing.GerryFairClassifier(),
                     inprocessing.GridSearchReduction(sklearn.linear_model.LogisticRegression(), constraints="DemographicParity", drop_prot_attr=False),
                     inprocessing.MetaFairClassifier(),
                     inprocessing.PrejudiceRemover(),
                     None]
postprocessing_algos = [postprocessing.calibrated_eq_odds_postprocessing.CalibratedEqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups),
                        postprocessing.RejectOptionClassification(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups),
                        #postprocessing.EqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups),
                        None]

for post in postprocessing_algos:
    for inproc in inprocessing_algos:
        for pre in preprocessing_algos:
            try:
                df = analyze_algo(dataset_train, dataset_test, privileged_groups, unprivileged_groups, 
                         preprocessing_algo=copy.deepcopy(pre), 
                         inprocessing_algo=copy.deepcopy(inproc),
                         postprocessing_algo=copy.deepcopy(post))
                display(df)
            except KeyboardInterrupt:
                break
            #except:
            #    print("FAILED: " + get_model_name(pre) + ", " + get_model_name(inproc) + ", " + get_model_name(post) + " on dataset " + get_dataset_name(dataset))
    

AttributeError: 'float' object has no attribute 'reshape'

<IPython.core.display.Javascript object>

# Disregard everything underneath

In [ ]:
%%notify
dataset = CompasDataset()
privileged_groups = [{'race': 1}]
unprivileged_groups = [{'race': 0}]
'''dataset = GermanDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['personal_status', 'sex'] #ignore sex-related stuff
)
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]'''
'''dataset = BankDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['day_of_week'] #ignore sex-related stuff
)
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]'''
'''dataset = AdultDataset()
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]'''

dataset_train, dataset_test = dataset.split([0.7], shuffle = True)


base_model = sklearn.linear_model.LogisticRegression()
base_metrics = run_base(dataset_train, dataset_test, base_model)

df = pd.DataFrame(base_metrics, columns=["Logistic Regression"])
df.index = ["accuracy", "theil index", "consistency", "false positive rate difference",
            "false negative rate difference", "accuracy difference",
            "false discovery rate difference", "false omission rate difference",
            "stat parity difference", "priv base rate", "unpriv base rate"]

RW = preprocessing.Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
fair_dataset_train = RW.fit_transform(dataset_train.copy())

fair_base_model = sklearn.linear_model.LogisticRegression()
fair_base_model.fit(fair_dataset_train.features, fair_dataset_train.labels.ravel())
results = fair_base_model.predict(dataset_test.features)
dataset_test_pred = dataset_test.copy()
dataset_test_pred.labels = np.array([results]).transpose()

EOP = postprocessing.EqOddsPostprocessing(unprivileged_groups=unprivileged_groups,
                                                     privileged_groups=privileged_groups)


CM = ClassificationMetric(dataset_test, EOP.fit_predict(dataset_test, dataset_test_pred), unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

df["RW + EOP"] = run_classification_metrics(CM)

''''''
DIR = preprocessing.DisparateImpactRemover()
fair_dataset_train = DIR.fit_transform(dataset_train.copy())

fair_base_model = sklearn.linear_model.LogisticRegression()
fair_base_model.fit(fair_dataset_train.features, fair_dataset_train.labels.ravel())
results = fair_base_model.predict(dataset_test.features)
dataset_test_pred = dataset_test.copy()
dataset_test_pred.labels = np.array([results]).transpose()

EOP = postprocessing.EqOddsPostprocessing(unprivileged_groups=unprivileged_groups,
                                                     privileged_groups=privileged_groups)


CM = ClassificationMetric(dataset_test, EOP.fit_predict(dataset_test, dataset_test_pred), unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
df["DIR + EOP"] = run_classification_metrics(CM)

''''''
'''optim_options = get_optim_preproc_options_dict(dataset)
OP = OptimPreproc(OptTools, optim_options,
              unprivileged_groups = unprivileged_groups,
              privileged_groups = privileged_groups)
fair_dataset_train = OP.fit_transform(dataset_train.copy())

fair_base_model = RandomForestClassifier(n_estimators=1100)
fair_base_model.fit(fair_dataset_train.features, fair_dataset_train.labels.ravel())
results = fair_base_model.predict(dataset_test.features)
dataset_test_pred = dataset_test.copy()
dataset_test_pred.labels = np.array([results]).transpose()

EOP = postprocessing.EqOddsPostprocessing(unprivileged_groups=unprivileged_groups,
                                                     privileged_groups=privileged_groups)


CM = ClassificationMetric(dataset_test, EOP.fit_predict(dataset_test, dataset_test_pred), unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
BLDM = BinaryLabelDatasetMetric(fair_dataset_train,
                                unprivileged_groups=unprivileged_groups,
                                privileged_groups=privileged_groups)
df["OP + EOP"] = np.concatenate((run_classification_metrics(CM), run_binary_dataset_metrics(BLDM)))'''
df

In [ ]:
dataset = GermanDataset(
    protected_attribute_names=['age'],
    privileged_classes=[lambda x: x >= 25], #age >= 25 is privileged
    features_to_drop=['personal_status', 'sex'] #ignore sex-related stuff
)
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

#dataset.labels = dataset.labels%2

preprocessing_algos = [preprocessing.Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups),
                      preprocessing.DisparateImpactRemover(),
                      None]
inprocessing_algos = [inprocessing.ExponentiatedGradientReduction(sklearn.linear_model.LogisticRegression(), constraints="DemographicParity", drop_prot_attr=False),
                     inprocessing.GerryFairClassifier(),
                     inprocessing.GridSearchReduction(sklearn.linear_model.LogisticRegression(), constraints="DemographicParity", drop_prot_attr=False),
                     inprocessing.MetaFairClassifier(),
                     inprocessing.PrejudiceRemover(),
                     None]
postprocessing_algos = [postprocessing.EqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups),
                       None]

pre = preprocessing.Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
inproc = inprocessing.GerryFairClassifier()
post = postprocessing.EqOddsPostprocessing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

df = analyze_algo(dataset_train, dataset_test, privileged_groups, unprivileged_groups, 
                     #preprocessing_algo=copy.deepcopy(pre), 
                     inprocessing_algo=inproc,
                     #postprocessing_algo=post
                 )
display(df)
